# Utilisation de l'augmentation des données pour améliorer les performances avec les API tf.data et tf.image.

L'augmentation de données est une technique puissante que nous pouvons appliquer pour incrémenter artificiellement la taille de notre ensemble de données, en créant des copies légèrement modifiées des images à notre disposition. Dans cette recette, nous tirerons parti des API tf.data et tf.image pour augmenter les performances d'un CNN formé sur le jeu de données difficile Caltech 101

Dans cette recette, nous utiliserons le jeu de données Caltech 101, disponible ici : http://www.vision.caltech.edu/Image_Datasets/Caltech101/. Téléchargez et décompressez 101_ObjectCategories.tar.gz à votre emplacement préféré.

https://drive.google.com/u/0/uc?export=download&confirm=k34b&id=137RyRjvTBkBiIfeYBNZBtViDHQ6_Ewsp

In [6]:
#!wget -c http://www.vision.caltech.edu/Image_Datasets/Caltech101/101_ObjectCategories.tar.gz

In [7]:
#!tar xvf 101_ObjectCategories.tar.gz

**1.** Importez les modules requis :

In [8]:
#!pip install git+https://github.com/tensorflow/docs

In [19]:
import os
import pathlib

import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import tensorflow_docs as tfdocs
import tensorflow_docs.plots
from glob import glob
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model

**2.** Créez un alias pour le  tf.data.experimental.AUTOTUNE, que nous utiliserons plus tard :

In [21]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

**3.** Dénissez une fonction pour créer une version plus petite de VGG. Commencez par créer la couche d'entrée et le premier bloc de deux convolutions de 32 filtres chacune :

In [20]:
def build_network(width, height, depth, classes):
    input_layer = Input(shape=(width, height, depth))

    x = Conv2D(filters=32,
               kernel_size=(3, 3),
               padding='same')(input_layer)
    x = ReLU()(x)
    x = BatchNormalization(axis=-1)(x)
    x = Conv2D(filters=32,
               kernel_size=(3, 3),
               padding='same')(x)
    x = ReLU()(x)
    x = BatchNormalization(axis=-1)(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(rate=0.25)(x)

    x = Conv2D(filters=64,
               kernel_size=(3, 3),
               padding='same')(x)
    x = ReLU()(x)
    x = BatchNormalization(axis=-1)(x)
    x = Conv2D(filters=64,
               kernel_size=(3, 3),
               padding='same')(x)
    x = ReLU()(x)
    x = BatchNormalization(axis=-1)(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(rate=0.25)(x)

    x = Flatten()(x)
    x = Dense(units=512)(x)
    x = ReLU()(x)
    x = BatchNormalization(axis=-1)(x)
    x = Dropout(rate=0.5)(x)

    x = Dense(units=classes)(x)
    output = Softmax()(x)

    return Model(input_layer, output)

**4.** Dénissez une fonction pour charger toutes les images de l'ensemble de données, ainsi que leurs étiquettes, en fonction de leurs chemins de fichiers :

In [23]:
def load_image_and_label(image_path, target_size=(64, 64)):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.convert_image_dtype(image, np.float32)
    image = tf.image.resize(image, target_size)

    label = tf.strings.split(image_path, os.path.sep)[-2]
    label = (label == CLASSES)  # One-hot encode.
    label = tf.dtypes.cast(label, tf.float32)

    return image, label

**5.** Déﬁnissez une fonction pour tracer et enregistrer la courbe d'entraînement d'un modèle :

In [22]:
def plot_model_history(model_history, metric, plot_name):
    plt.style.use('seaborn-darkgrid')
    plotter = tfdocs.plots.HistoryPlotter()
    plotter.plot({'Model': model_history}, metric=metric)

    plt.title(f'{metric.upper()}')
    plt.ylim([0, 1])

    plt.savefig(f'{plot_name}.png')
    plt.close()

**6.** Déﬁnissez une fonction pour augmenter une image en effectuant des transformations aléatoires dessus :

In [24]:
def augment(image, label):
    image = tf.image.resize_with_crop_or_pad(image, 74, 74)
    image = tf.image.random_crop(image, size=(64, 64, 3))
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_brightness(image, 0.2)

    return image, label

**7.** Dénissez une fonction pour préparer un tf.data.Dataset d'images, basé sur un modèle de type glob qui fait référence au dossier dans lequel elles résident :

In [25]:
def prepare_dataset(data_pattern):
    return (tf.data.Dataset
            .from_tensor_slices(data_pattern)
            .map(load_image_and_label,
                 num_parallel_calls=AUTOTUNE))

**8.** Définissez la graine aléatoire :

In [10]:
SEED = 999
np.random.seed(SEED)

**9.** Chargez les chemins vers toutes les images de l'ensemble de données, à l'exception de celles de la classe BACKGROUND_Google :

In [26]:
base_path = (pathlib.Path('/content') / '101_ObjectCategories')
images_pattern = str(base_path / '*' / '*.jpg')
image_paths = [*glob(images_pattern)]
image_paths = [p for p in image_paths if
               p.split(os.path.sep)[-2] != 'BACKGROUND_Google']

**10
.** Calculez les catégories uniques dans l'ensemble de données :

In [27]:
CLASSES = np.unique([p.split(os.path.sep)[-2]
                     for p in image_paths])

**11.** Divisez les chemins d'images en sous-ensembles d'entraînement et de test :

In [28]:
train_paths, test_paths = train_test_split(image_paths,
                                           test_size=0.2,
                                           random_state=SEED)

**12.** Préparez les ensembles de données d'entraînement et de test, sans augmentation :

In [29]:
BATCH_SIZE = 64
BUFFER_SIZE = 1024
train_dataset = (prepare_dataset(train_paths)
                 .batch(BATCH_SIZE)
                 .shuffle(buffer_size=BUFFER_SIZE)
                 .prefetch(buffer_size=BUFFER_SIZE))
test_dataset = (prepare_dataset(test_paths)
                .batch(BATCH_SIZE)
                .prefetch(buffer_size=BUFFER_SIZE))

**13.** Instancier, compiler, former et évaluer le réseau :

In [30]:
EPOCHS = 40
model = build_network(64, 64, 3, len(CLASSES))
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])
history = model.fit(train_dataset,
                    validation_data=test_dataset,
                    epochs=EPOCHS)
result = model.evaluate(test_dataset)
print(f'Test accuracy: {result[1]}')
plot_model_history(history, 'accuracy', 'normal')

Epoch 1/40
109/109 [==============================] - 16s 74ms/step - loss: 2.9201 - accuracy: 0.4106 - val_loss: 4.8998 - val_accuracy: 0.0847
Epoch 2/40
109/109 [==============================] - 14s 71ms/step - loss: 1.6160 - accuracy: 0.6243 - val_loss: 4.8924 - val_accuracy: 0.0806
Epoch 3/40
109/109 [==============================] - 14s 71ms/step - loss: 0.9280 - accuracy: 0.7774 - val_loss: 3.5355 - val_accuracy: 0.2851
Epoch 4/40
109/109 [==============================] - 14s 72ms/step - loss: 0.4874 - accuracy: 0.8960 - val_loss: 1.9808 - val_accuracy: 0.5593
Epoch 5/40
109/109 [==============================] - 14s 72ms/step - loss: 0.2633 - accuracy: 0.9519 - val_loss: 1.5918 - val_accuracy: 0.6382
Epoch 6/40
109/109 [==============================] - 14s 72ms/step - loss: 0.1447 - accuracy: 0.9797 - val_loss: 1.7000 - val_accuracy: 0.6094
Epoch 7/40
109/109 [==============================] - 14s 72ms/step - loss: 0.0950 - accuracy: 0.9849 - val_loss: 1.6153 - val_accuracy:

**14.** Préparez les ensembles d'entraînement et de test, cette fois en appliquant l'augmentation de données à l'ensemble d'entraînement :

In [31]:
train_dataset = (prepare_dataset(train_paths)
                 .map(augment, num_parallel_calls=AUTOTUNE)
                 .batch(BATCH_SIZE)
                 .shuffle(buffer_size=BUFFER_SIZE)
                 .prefetch(buffer_size=BUFFER_SIZE))
test_dataset = (prepare_dataset(test_paths)
                .batch(BATCH_SIZE)
                .prefetch(buffer_size=BUFFER_SIZE))

**15.** Instanciez, compilez, entraînez et évaluez le réseau sur les données augmentées :

In [32]:
model = build_network(64, 64, 3, len(CLASSES))
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])
history = model.fit(train_dataset,
                    validation_data=test_dataset,
                    epochs=EPOCHS)
result = model.evaluate(test_dataset)
print(f'Test accuracy: {result[1]}')
plot_model_history(history, 'accuracy', 'augmented')

Epoch 1/40
109/109 [==============================] - 16s 72ms/step - loss: 3.6235 - accuracy: 0.3037 - val_loss: 6.3118 - val_accuracy: 0.0121
Epoch 2/40
109/109 [==============================] - 14s 69ms/step - loss: 2.6895 - accuracy: 0.4216 - val_loss: 6.7092 - val_accuracy: 0.0190
Epoch 3/40
109/109 [==============================] - 14s 70ms/step - loss: 2.2102 - accuracy: 0.4949 - val_loss: 4.4150 - val_accuracy: 0.1861
Epoch 4/40
109/109 [==============================] - 14s 70ms/step - loss: 1.8942 - accuracy: 0.5558 - val_loss: 2.3251 - val_accuracy: 0.4643
Epoch 5/40
109/109 [==============================] - 14s 70ms/step - loss: 1.7065 - accuracy: 0.5905 - val_loss: 1.7374 - val_accuracy: 0.5893
Epoch 6/40
109/109 [==============================] - 14s 70ms/step - loss: 1.5330 - accuracy: 0.6240 - val_loss: 1.5593 - val_accuracy: 0.6267
Epoch 7/40
109/109 [==============================] - 14s 69ms/step - loss: 1.3972 - accuracy: 0.6512 - val_loss: 1.6511 - val_accuracy:

Nous venons d'implémenter une version allégée de la célèbre architecture VGG, entraînée sur le jeu de données Caltech 101. Pour mieux comprendre les avantages de l'augmentation de données, nous avons installé une première version sur les données d'origine, sans aucune modication, obtenant un niveau de précision de 64,45 % sur l'ensemble de test. Ce premier modèle montre des signes de dépassement, car l'écart qui sépare les courbes de précision d'apprentissage et de validation s'élargit au début du processus d'apprentissage.

Ensuite, nous avons entraîné le même réseau sur un jeu de données augmenté, en utilisant la fonction augment() définie précédemment. Cela a grandement amélioré les performances du modèle, atteignant une précision respectable de 73,55% sur l'ensemble de test. De plus, l'écart entre les courbes de précision d'entraînement et de validation est sensiblement plus petit, ce qui suggère un effet de régularisation résultant de l'application de l'augmentation des données.



